In [1]:
import pandas as pd
import requests
import json
from datetime import datetime
import time
import math

### TVL de las principales chains using API Defi-llama

In [2]:
url_chain =  "https://api.llama.fi/v2/historicalChainTvl/"
chains = ['Ethereum', 'Solana', 'BSC', 'Tron', 'Tron', 'Sui', 'Aptos']
dfs = []

for chain in chains:
    url = url_chain + chain

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)

        df['date'] = pd.to_datetime(df['date'], unit='s')
        df.set_index('date', inplace=True)

        df.rename(columns={'tvl':chain}, inplace=True)
        dfs.append(df)
    else:
        print(f'Error al realizar la solicitud. Código de estado: {response.status_code}')

df = pd.concat(dfs, axis=1)
df.head(5)

,Ethereum,Solana,BSC,Tron,Tron,Sui,Aptos
date,,,,,,,
2018-04-27,404077.858810,NaN,NaN,NaN,NaN,NaN,NaN
2018-04-28,400585.099213,NaN,NaN,NaN,NaN,NaN,NaN
2018-04-29,429213.933200,NaN,NaN,NaN,NaN,NaN,NaN
2018-04-30,421651.156158,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-01,436194.969783,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
import plotly.express as px

df_long = df.reset_index(names="date").melt(id_vars="date", var_name="Chain", value_name="TVL")

# Graficar
fig = px.line(df_long, x="date", y="TVL", color="Chain", title="Chain TVL")
fig.update_layout(template="plotly_dark")

fig.show()


### L2 Ethereum

In [4]:
base_url = "https://api.llama.fi/v2/historicalChainTvl/"
l2s = ['Base', 'Arbitrum', 'Taiko', 'Optimism', 'Blast', 'Mantle',
         'Linea', 'Scroll', 'ZKsync Era']
dfs = []

for l2 in l2s:
    url = base_url + l2
    response = requests.get(url)
   
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)
        
        df['date'] = pd.to_datetime(df['date'], unit='s')
        df.set_index('date', inplace=True)
        df.rename(columns={'tvl':l2}, inplace=True)
        dfs.append(df)
    
    else:
        print(f'Error al realizar la solicitud. Código de estado: {response.status_code}')

df = pd.concat(dfs, axis=1)
df = df[df.index >= '2024-01-01']
df.head()


,Base,Arbitrum,Taiko,Optimism,Blast,Mantle,Linea,Scroll,ZKsync Era
date,,,,,,,,,
2024-01-01,4.386885e+08,2.326316e+09,NaN,8.952263e+08,NaN,1.215214e+08,5.336343e+07,2.345125e+07,1.617451e+08
2024-01-02,4.457479e+08,2.416401e+09,NaN,9.220480e+08,NaN,1.252580e+08,5.516133e+07,2.398387e+07,1.669485e+08
2024-01-03,4.396513e+08,2.414302e+09,NaN,9.162564e+08,NaN,1.233165e+08,5.472998e+07,2.409044e+07,1.659129e+08
2024-01-04,4.222769e+08,2.313545e+09,NaN,8.610313e+08,NaN,1.186131e+08,5.293274e+07,2.354751e+07,1.592095e+08
2024-01-05,4.228744e+08,2.348421e+09,NaN,8.640763e+08,NaN,1.220167e+08,5.310664e+07,2.444492e+07,1.619001e+08


In [5]:
import plotly.express as px

fig = px.line(df, x=df.index, y=df.columns, title="TVL L2")
fig.update_layout(template='plotly_dark')

fig.show()

---

### Variación porcentual chains TVL

In [6]:
baseUrl = 'https://api.llama.fi/v2/historicalChainTvl/'
chains = ['Ethereum', 'Solana', 'BSC', 'Tron', 'Sui', 'Aptos' ,'Base', 'Arbitrum', 'Polygon', 'Optimism', 'Blast', 'Mantle', 'Linea', 'Scroll']
dfs = []

In [7]:
for chain in chains:
    url = base_url + chain
    response = requests.get(url)

    if response.status_code ==200:
        data = response.json()
        df = pd.DataFrame(data)
        df['date'] = pd.to_datetime(df['date'], unit='s')
        df.set_index('date', inplace=True)
        df.rename(columns={'tvl':chain}, inplace=True)

        #calcular cambios porcentuales
        df['%1D'] = df[chain].pct_change(1) * 100
        df['%7D'] = df[chain].pct_change(7) * 100
        df['%30D'] = df[chain].pct_change(30) * 100

        last_row = df.iloc[-1][['%1D', '%7D', '%30D']]
        last_row.name = chain
        dfs.append(last_row)
    else:
        print(f'Error al realizar la solicitud. Código de estado: {response.status_code}')
    

df = pd.concat(dfs, axis=1)
df.T

,%1D,%7D,%30D
Ethereum,-0.947750,-0.231910,22.055023
Solana,1.912935,15.798434,45.164080
BSC,-2.550032,1.957499,10.407903
Tron,3.204322,12.700153,12.657478
Sui,-1.154244,16.813328,52.011353
Aptos,-4.182774,-5.470985,27.464581
Base,-0.243294,7.364137,28.141384
Arbitrum,-0.557129,1.255130,17.406786
Polygon,-0.493439,-8.756298,-1.448467
Optimism,-0.530214,-2.507955,9.798071


---

### MARK CAP memecoin > 500M

In [8]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

ModuleNotFoundError: No module named 'pycoingecko'

In [ ]:
coinsMktDataByCategory = cg.get_coins_markets(vs_currency = 'usd',
                                              category = 'meme-token',
                                               order = 'volume_desc')

coinsMktDataByCategory = pd.DataFrame(coinsMktDataByCategory)
coinsMktDataByCategory.columns

Index(['id', 'symbol', 'name', 'image', 'current_price', 'market_cap',
       'market_cap_rank', 'fully_diluted_valuation', 'total_volume',
       'high_24h', 'low_24h', 'price_change_24h',
       'price_change_percentage_24h', 'market_cap_change_24h',
       'market_cap_change_percentage_24h', 'circulating_supply',
       'total_supply', 'max_supply', 'ath', 'ath_change_percentage',
       'ath_date', 'atl', 'atl_change_percentage', 'atl_date', 'roi',
       'last_updated'],
      dtype='object')

In [ ]:
df = coinsMktDataByCategory[['name','symbol', 'current_price', 'market_cap', 'price_change_24h']]
#df =df[df['market_cap'] >= 200000000]
df = df.loc[df['market_cap'] >=500000000 ]
df

,name,symbol,current_price,market_cap,price_change_24h
0,Dogecoin,doge,0.361052,5.299386e+10,-1.699083e-03
1,Pepe,pepe,0.000021,8.819512e+09,-3.257239e-07
2,Bonk,bonk,0.000051,3.494760e+09,4.520000e-06
3,Peanut the Squirrel,pnut,1.870000,1.871537e+09,1.875410e-01
4,Shiba Inu,shib,0.000024,1.435545e+10,-2.874506e-08
5,dogwifhat,wif,3.590000,3.582838e+09,-1.157005e-03
6,FLOKI,floki,0.000256,2.470706e+09,9.300000e-06
7,BOOK OF MEME,bome,0.010449,7.207411e+08,3.655700e-04
8,Neiro,neiro,0.002170,9.128487e+08,1.474300e-04
9,Act I The AI Prophecy,act,0.664558,6.239795e+08,-3.087821e-03


In [ ]:
fig = px.pie(
    df, 
    values='market_cap', 
    names='symbol', 
    title="Market Cap Memecoins",
    color_discrete_sequence=px.colors.qualitative.Pastel  # Colores suaves para mejor visualización
)
fig.update_layout(
    showlegend=True,
    legend_title_text='Criptomonedas',
    margin=dict(t=50, b=50, l=25, r=25),
    template='plotly_dark')

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

---

### Volumen del día de hoy en los principales exchanges

In [ ]:
import datetime as dt
ids= ['binance', 'coinbase-exchange']

exchangevolume = []

for exchnage in ids:
    exchgVolume = cg.get_exchanges_volume_chart_by_id(id=exchnage, days=1)
    exchgVolumeDataFrame = pd.DataFrame(exchgVolume,  columns = ['Date', exchnage])
    exchgVolumeDataFrame['Date'] = pd.to_datetime(exchgVolumeDataFrame['Date'], unit='ms')

    exchangevolume.append(exchgVolumeDataFrame.set_index('Date'))

df = pd.concat(exchangevolume, axis=1)
df.head()


,binance,coinbase-exchange
Date,,
2024-11-17 01:30:00,312521.8231667371816753,51294.8487832762423387
2024-11-17 01:40:00,313913.654495712561863,51351.2230875320727403
2024-11-17 01:50:00,316445.5724716980216563,51777.2474021889813727
2024-11-17 02:00:00,317350.693328501884409,51753.4729307190833923
2024-11-17 02:10:00,323546.4160631411755125,50357.9079764433120473


In [ ]:
df[['binance', 'coinbase-exchange']] = df[['binance', 'coinbase-exchange']].apply(pd.to_numeric)

df.head()

,binance,coinbase-exchange
Date,,
2024-11-17 01:30:00,312521.823167,51294.848783
2024-11-17 01:40:00,313913.654496,51351.223088
2024-11-17 01:50:00,316445.572472,51777.247402
2024-11-17 02:00:00,317350.693329,51753.472931
2024-11-17 02:10:00,323546.416063,50357.907976


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Agregar la línea para Binance en el eje primario
fig.add_trace(go.Scatter(x=df.index, y=df['binance'], mode='lines', name='binance', line=dict(color='yellow')))

# Agregar la línea para Coinbase en un eje secundario
fig.add_trace(go.Scatter(x=df.index, y=df['coinbase-exchange'], mode='lines', name='coinbase-exchange', line=dict(color='red'), yaxis="y2"))

# Configuración del diseño
fig.update_layout(
    title="Volumen Biance y coinbase Exchange",
    xaxis_title="Date",
    yaxis_title="Volumen Binance",
    yaxis2=dict(
        title="Volume Coinbase",
        overlaying='y',   # Superpone el eje secundario al eje primario
        side='right'      # Muestra el eje secundario en el lado derecho
    ),
    showlegend=True,
    legend_title_text="Exchanges",
    template="plotly_dark"
)

fig.show()